<a href="https://colab.research.google.com/github/aryaganu10-cpu/GenAi/blob/main/Task5_W2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import re

# Regex to find URLs at the end of the string. URLs can have various characters.
# This regex captures common URL patterns at the end of a string.
url_regex = r'(https?://(?:www\.)?[a-zA-Z0-9./_\-]+(?:\?[a-zA-Z0-9=&_\-]+)?)' # Matches http(s):// followed by non-space characters.

# Extract URLs into a new 'URL' column
financial_news_df['URL'] = financial_news_df['text'].str.extract(url_regex + r'$', expand=False)

# Remove URLs from the 'text' column
financial_news_df['text'] = financial_news_df['text'].str.replace(url_regex + r'$', '', regex=True)

# Display the updated DataFrame
display(financial_news_df.head())

,text,label,URL
0,Here are Thursday's biggest analyst calls: App...,0,https://t.co/QPN8Gwl7Uh
1,Buy Las Vegas Sands as travel to Singapore bui...,0,https://t.co/fLS2w57iCz
2,"Piper Sandler downgrades DocuSign to sell, cit...",0,https://t.co/1EmtywmYpr
3,"Analysts react to Tesla's latest earnings, bre...",0,https://t.co/kwhoE6W06u
4,Netflix and its peers are set for a ‘return to...,0,https://t.co/jPpdl0D9s4


In [1]:
import pandas as pd

financial_news_df = pd.read_csv('/content/financial_news.csv')
display(financial_news_df.head())

,text,label
0,Here are Thursday's biggest analyst calls: App...,0
1,Buy Las Vegas Sands as travel to Singapore bui...,0
2,"Piper Sandler downgrades DocuSign to sell, cit...",0
3,"Analysts react to Tesla's latest earnings, bre...",0
4,Netflix and its peers are set for a ‘return to...,0


# Task
Create a semantic search tool for the `financial_news_df` dataset using `sentence-transformers` and `gradio`. The tool should allow users to input a query, and it will return the top 5 most semantically similar news articles from the `financial_news_df` DataFrame, displaying the original text, label, and URL of each result in a Gradio interface.

## Install Libraries

### Subtask:
Install necessary Python libraries: 'sentence-transformers' for generating embeddings and 'gradio' for building the user interface.


**Reasoning**:
Install the 'sentence-transformers' library using pip to enable the generation of embeddings.



In [3]:
pip install sentence-transformers

**Reasoning**:
Install the 'gradio' library using pip to build the user interface.



In [4]:
pip install gradio

## Load Pre-trained Model

### Subtask:
Load a pre-trained sentence embedding model (e.g., 'all-MiniLM-L6-v2') from the 'sentence-transformers' library.


**Reasoning**:
Import the `SentenceTransformer` class and load the 'all-MiniLM-L6-v2' model to prepare for generating embeddings.



In [5]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

print("Model 'all-MiniLM-L6-v2' loaded successfully.")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model 'all-MiniLM-L6-v2' loaded successfully.


**Reasoning**:
The previous code cell successfully loaded the `all-MiniLM-L6-v2` model. The next logical step is to use this model to generate embeddings for the text data in the `financial_news_df` DataFrame.



In [6]:
sentences = financial_news_df['text'].tolist()
embeddings = model.encode(sentences, show_progress_bar=True)
financial_news_df['embedding'] = embeddings.tolist()

print("Embeddings generated and added to 'financial_news_df'.")
display(financial_news_df.head())

Batches:   0%|          | 0/531 [00:00<?, ?it/s]

Embeddings generated and added to 'financial_news_df'.


,text,label,URL,embedding
0,Here are Thursday's biggest analyst calls: App...,0,https://t.co/QPN8Gwl7Uh,"[-0.0014813962625339627, -0.058853961527347565..."
1,Buy Las Vegas Sands as travel to Singapore bui...,0,https://t.co/fLS2w57iCz,"[0.020164918154478073, -0.020287901163101196, ..."
2,"Piper Sandler downgrades DocuSign to sell, cit...",0,https://t.co/1EmtywmYpr,"[-0.027000561356544495, -0.022857561707496643,..."
3,"Analysts react to Tesla's latest earnings, bre...",0,https://t.co/kwhoE6W06u,"[-0.06276890635490417, 0.0457136370241642, 0.0..."
4,Netflix and its peers are set for a ‘return to...,0,https://t.co/jPpdl0D9s4,"[-0.03898603096604347, -0.1573472023010254, -0..."


**Reasoning**:
With the embeddings generated, the next logical step is to define a function that will take a user query, convert it into an embedding using the same model, and then calculate the cosine similarity between the query embedding and the document embeddings to find the most similar articles. This function will be central to the semantic search tool.



In [7]:
from sklearn.metrics.pairwise import cosine_similarity

def semantic_search(query, model, financial_news_df, top_n=5):
    # Encode the query
    query_embedding = model.encode(query)

    # Calculate cosine similarity between query and all document embeddings
    # Reshape query_embedding for cosine_similarity function
    similarities = cosine_similarity([query_embedding], list(financial_news_df['embedding'].values))

    # Get the top_n similar articles
    top_indices = similarities.argsort()[0][::-1][:top_n]

    # Return the top_n articles
    results = financial_news_df.iloc[top_indices]
    return results

print("Semantic search function defined.")

Semantic search function defined.


**Reasoning**:
Now that the semantic search function is defined, the next step is to create a Gradio interface to allow users to interact with the search tool. This involves defining a wrapper function for Gradio that uses the `semantic_search` function and setting up the Gradio UI components.



In [8]:
import gradio as gr

def search_interface(query):
    results = semantic_search(query, model, financial_news_df)
    output_string = []
    for index, row in results.iterrows():
        output_string.append(f"**Text:** {row['text']}<br> **Label:** {row['label']}<br> **URL:** {row['URL']}\n---\n")
    return "".join(output_string)

# Create the Gradio interface
iface = gr.Interface(
    fn=search_interface,
    inputs=gr.Textbox(lines=2, placeholder="Enter your search query here..."),
    outputs="html",
    title="Semantic Search Tool for Financial News",
    description="Enter a query to find the top 5 most semantically similar news articles."
)

# Launch the interface
iface.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b22037d9b348fa4238.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b22037d9b348fa4238.gradio.live


# Task
Modify the Gradio launch command in the last code cell (`19278d87`) by removing the `debug=True` argument from `iface.launch()` to potentially speed up the interface startup, and then confirm that the Gradio interface has been relaunched with these new settings.

## Modify Gradio Launch

### Subtask:
Modify the existing Gradio launch command in the last code cell to remove the `debug=True` argument, which can sometimes speed up the interface startup.


**Reasoning**:
The subtask is to modify the `iface.launch()` command to remove the `debug=True` argument. I will provide a `code_block` to update the specific cell, replacing `iface.launch(debug=True)` with `iface.launch()`.



In [9]:
import gradio as gr

def search_interface(query):
    results = semantic_search(query, model, financial_news_df)
    output_string = []
    for index, row in results.iterrows():
        output_string.append(f"**Text:** {row['text']}<br> **Label:** {row['label']}<br> **URL:** {row['URL']}\n---\n")
    return "".join(output_string)

# Create the Gradio interface
iface = gr.Interface(
    fn=search_interface,
    inputs=gr.Textbox(lines=2, placeholder="Enter your search query here..."),
    outputs="html",
    title="Semantic Search Tool for Financial News",
    description="Enter a query to find the top 5 most semantically similar news articles."
)

# Launch the interface
iface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4d921be0504aa4b634.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Final Task

### Subtask:
Confirm that the Gradio interface has been relaunched with potentially faster startup settings.


## Summary:

### Data Analysis Key Findings
*   The `debug=True` argument was successfully removed from the `iface.launch()` command as intended.
*   Gradio automatically configured `share=True` upon launch, generating a public URL for accessing the interface, due to the execution environment being a hosted Jupyter notebook (Colab).
*   The Gradio interface was successfully relaunched and became accessible via the provided public URL.
*   An informational message from Gradio was displayed, advising to set `debug=True` explicitly in `launch()` for Colab notebooks to show errors, which is a general recommendation and not an error related to the modification.

### Insights or Next Steps
*   The Gradio interface is now launched without the `debug=True` flag, which can lead to potentially faster startup times and a production-like behavior.
*   The interface is successfully deployed and accessible via a public URL, ready for external testing or usage.
